In [1]:
import os
from github import Github
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm
import time
import json
from multiprocessing.pool import ThreadPool
from github.GithubException import RateLimitExceededException
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import datetime
import json
import pinecone    
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, Language
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone

/Users/jvh/Desktop/codemate/ingestion/.venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Load all repos depending on FastAPI

In [3]:

repo = "tiangolo/fastapi"
url = 'https://github.com/{}/network/dependents'.format(repo)
nextExists = True
result = []
while nextExists:
    try: 
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        result = result + [
            "{}/{}".format(
                t.find('a', {"data-repository-hovercards-enabled":""}).text,
                t.find('a', {"data-hovercard-type":"repository"}).text
            )
            for t in soup.findAll("div", {"class": "Box-row"})
        ]
        nextExists = False
        for u in soup.find("div", {"class":"paginate-container"}).findAll('a'):
            if u.text == "Next":
                nextExists = True
                url = u["href"]
    except:
        print("Error")

    print(f"Has {len(result)} dependents")
    if len(result) > 5000:
        break

    #time.sleep(1)




Has 30 dependents
Has 60 dependents
Has 90 dependents
Has 120 dependents
Has 150 dependents
Has 180 dependents
Has 210 dependents
Has 240 dependents
Has 270 dependents
Has 300 dependents
Has 330 dependents
Has 360 dependents
Has 390 dependents
Has 420 dependents
Has 450 dependents
Has 480 dependents
Has 510 dependents
Has 540 dependents
Has 570 dependents
Has 600 dependents
Has 630 dependents
Has 660 dependents
Has 690 dependents
Has 720 dependents
Has 750 dependents
Has 780 dependents
Has 810 dependents
Has 840 dependents
Has 869 dependents
Has 899 dependents
Has 929 dependents
Has 959 dependents
Has 989 dependents
Has 1019 dependents
Has 1049 dependents
Has 1079 dependents
Has 1109 dependents
Has 1139 dependents
Has 1169 dependents
Has 1199 dependents
Has 1229 dependents
Has 1259 dependents
Has 1289 dependents
Has 1319 dependents
Has 1348 dependents
Has 1378 dependents
Has 1408 dependents
Has 1438 dependents
Has 1468 dependents
Has 1498 dependents
Has 1528 dependents
Has 1558 depende

In [4]:
with open("data/.meta/dependents.json", "w") as f:
    json.dump(result, f, indent=2)

In [7]:
with open("data/.meta/dependents.json", "r") as f:
    result = json.load(f)

In [8]:
print(len(result))
print(result)

3567
['HeavY-Futhark/EmoBot', 'olajide1234/nurfa', 'InteliProjects/2023M7T1-Inteli-Grupo-5', 'ClockGU/time-vault', 'Ipsitaaaa/Sentiment-Analysis', 'natalialuizads/guardiao-financeiro-api', 'KELONMYOSA/dl-course-cv', 'mysociety/twfy-votes', 'Ishsshukla/course_2_ml', 'quillan86/gambrinus', 'RolletQuentin/teasing-solver', 'Alfred910226/ptt-article-recommender-system', 'arunchandra23/tensorGo-assignment', 'gireeshkbogu/comic_grill', 'AI-Tarot/app-backend', 'mathis-blln/projet_info', 'Ayushithakur-2003/testapi', 'satyamkrchhetri/Sentiment-Analysis', 'yash-fetchai/whatsapp-webhook12', 'Ishatvb/Potato_Leaf_Disease_Detection_Appliction', 'krasulna/FormInspector', 'loupiote03/openshift_training', 'oliverxleecomcast/02_04_ci_for_python', 'nknguyenhc/NeuralCats', 'aguiarpaulo/api_project', 'hirebbec/web_chat_v2', 'ShirvanyanKaren/stock-analytics-pern', 'omsandipatil/RVC2', 'rafaelpierre/pitchgpt', 'elvishabib/Life-TL221-Challenge', 'wiretick/dopom', 'SueRix/API_Gateway_and_Auth', 'dkarduner/pytho

### Get details of the found repos

In [ ]:
github_api_key = os.environ.get('GITHUB_API_KEY')
assert github_api_key is not None, "You need to set your GITHUB_API_KEY environment variable."
g = Github(github_api_key, retry=None)

In [19]:
def get_repo(name):
    try: 
        repo = g.get_repo(name)
        return {
            'id': name,
            'stars': repo.stargazers_count,
            'forks': repo.forks_count,
            'url': repo.html_url,
        }
        time.sleep(1)
    except RateLimitExceededException as e:
        print("Rate limit exceeded, retry at:", datetime.datetime.fromtimestamp(int(e.headers['x-ratelimit-reset'])))
    except Exception as e:
        print(f"Error downloading {name}:", e)
        return None

get_repo("tiangolo/fastapi")

Rate limit exceeded, retry at: 2023-11-17 17:35:45


In [69]:
print("Getting repos")
pool = ThreadPool(3)
repos = list(tqdm(pool.imap(get_repo, result), total=len(result)))
repos = [r for r in repos if r is not None]
repos_df = pd.DataFrame(repos)

Getting repos


  0%|          | 14/3567 [00:02<09:51,  6.00it/s]

Error downloading gireeshkbogu/comic_grill: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}


 29%|██▉       | 1030/3567 [03:14<07:58,  5.30it/s]


NameError: name 'datetime' is not defined

In [13]:
if repos is None:
    repos = [
        {
            "id": "HeavY-Futhark/EmoBot",
            "stars": 0,
            "forks": 0,
            "url": "https://github.com/HeavY-Futhark/EmoBot",
            #'repo': Repository(full_name="HeavY-Futhark/EmoBot")
        },
        {
            "id": "olajide1234/nurfa",
            "stars": 0,
            "forks": 0,
            "url": "https://github.com/olajide1234/nurfa",
            #'repo': Repository(full_name="olajide1234/nurfa"
        },
        {
            "id": "InteliProjects/2023M7T1-Inteli-Grupo-5",
            "stars": 0,
            "forks": 0,
            "url": "https://github.com/InteliProjects/2023M7T1-Inteli-Grupo-5",
            #'repo': Repository(full_name="InteliProjects/2023M7T1-Inteli-Grupo-5"
        },
    ]
    repos_df = pd.DataFrame(repos)

In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(repos_df.sort_values(by=['stars'], ascending=False).head(50))

,id,stars,forks,url
0,HeavY-Futhark/EmoBot,0,0,https://github.com/HeavY-Futhark/EmoBot
1,olajide1234/nurfa,0,0,https://github.com/olajide1234/nurfa
2,InteliProjects/2023M7T1-Inteli-Grupo-5,0,0,https://github.com/InteliProjects/2023M7T1-Int...


In [15]:
repos_df.sort_values(by=['stars'], ascending=False).head(10)

,id,stars,forks,url
0,HeavY-Futhark/EmoBot,0,0,https://github.com/HeavY-Futhark/EmoBot
1,olajide1234/nurfa,0,0,https://github.com/olajide1234/nurfa
2,InteliProjects/2023M7T1-Inteli-Grupo-5,0,0,https://github.com/InteliProjects/2023M7T1-Int...


### Download and unzip these repos

In [18]:
def download_and_unzip(url, extract_to):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

def download_repo(repo_id, extract_to):
    try: 
        # Automatically downloads main if master doesn't exist
        download_and_unzip(f"https://github.com/{repo_id}/archive/refs/heads/master.zip", extract_to=extract_to)
    except:
        print(f"Error downloading repo {repo_id}")

os.makedirs("data", exist_ok=True)
for i, row in tqdm(repos_df.iterrows()): 
    download_repo(row['id'], extract_to="data")

3it [00:22,  7.55s/it]


### Ingest the files

In [2]:
all_files = []

for root, dirs, files in os.walk("data", topdown=True):
   if root in ['data/.meta', 'data']:
      continue

   files = [os.path.join(root, f) for f in files if f.endswith('.py') or f.endswith('.md')]
   all_files += files

len(all_files), all_files[:5]

(1848,
 ['data/nurfa-main/README.md',
  'data/nurfa-main/tests/__init__.py',
  'data/nurfa-main/nurfa/__init__.py',
  'data/nurfa-main/nurfa/consts.py',
  'data/nurfa-main/nurfa/main.py'])

In [3]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY", None)
assert pinecone_api_key is not None, "Needs PINECONE_API_KEY"
pinecone.init(      
	api_key=pinecone_api_key,      
	environment='gcp-starter'      
)      
index = pinecone.Index('codemate')

In [4]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=400, chunk_overlap=100
)

In [5]:
texts = []
for file in all_files: 
    with open(file) as f: 
        file_content = f.read()
        if file_content:
            texts.append(Document(page_content=file_content, metadata={"filename": file}))

texts[0]

Document(page_content='from datetime import datetime, timedelta\nfrom typing import Literal\n\n\nPossibleCategory = Literal[\n    "business", "entertainment", "general", "health", "science", "sports", "technology"\n]\n\nPossibleCountry = Literal[\n    "ae",\n    "ar",\n    "at",\n    "au",\n    "be",\n    "bg",\n    "br",\n    "ca",\n    "ch",\n    "cn",\n    "co",\n    "cu",\n    "cz",\n    "de",\n    "eg",\n    "fr",\n    "gb",\n    "gr",\n    "hk",\n    "hu",\n    "id",\n    "ie",\n    "il",\n    "in",\n    "it",\n    "jp",\n    "kr",\n    "lt",\n    "lv",\n    "ma",\n    "mx",\n    "my",\n    "ng",\n    "nl",\n    "no",\n    "nz",\n    "ph",\n    "pl",\n    "pt",\n    "ro",\n    "rs",\n    "ru",\n    "sa",\n    "se",\n    "sg",\n    "si",\n    "sk",\n    "th",\n    "tr",\n    "tw",\n    "vu",\n    "av",\n    "us",\n    "ve",\n    "za",\n]\n\nPossibleSearchIn = Literal[\n    "title",\n    "description",\n    "content",\n    "title,description",\n    "title,content",\n    "descriptio

In [6]:
chunks = text_splitter.split_documents(texts)

In [7]:
len(chunks), chunks[0]

(28347,
 Document(page_content='from datetime import datetime, timedelta\nfrom typing import Literal\n\n\nPossibleCategory = Literal[\n    "business", "entertainment", "general", "health", "science", "sports", "technology"\n]', metadata={'filename': 'data/nurfa-main/nurfa/consts.py'}))

In [ ]:
embeddings = OpenAIEmbeddings()

index_name = "codemate"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)

docsearch = Pinecone.from_documents(chunks, embeddings, index_name=index_name)